# Group Project / Assignment 3: Retrieval-Augmented Generation Question Answering
**Assignment due 6 April 11:59pm 2025**

Welcome to the third assignment for 50.055 Machine Learning Operations. 
The third and fourth assignment together form the course group project. You will be working in your project groups to build a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset. 
6. There is a maximum of 225 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission instructions are located at the bottom of the notebook.



### Retrieval-Augmented Generation (RAG) 

In this assignment, you will be building a Retrieval-Augmented Generation (RAG) question answering system which can answer questions about SUTD.

We'll be leveraging `langchain` and `llama 3.2`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)


The SUTD website used to allow chatting with current students. Unfortunately, this feature does not exist anymore. Let's build a chatbot to fill this gap!


### Conduct user research

What are the questions that prospective and current students have about SUTD? In week 2, you already conducted some user research to understand your users.

### Value Proposition Canvas


### QUESTION: 

Paste the value proposition canvas which you have created in week 2 into this notebook below. 


**--- ADD YOUR SOLUTION HERE (10 points) ---**

- (replace canvas image below)

------------------------------


![image.png](images/canvas.png)

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [ ]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**
! pip install -r requirements.txt

# For CUDA purposes, uncomment the following line and run it in your terminal:
#! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importing Libraries

In [2]:
import os
import json
import pickle
import re
from glob import glob
import openai
import faiss
import numpy as np
import pandas as pd
import torch
from transformers import pipeline
from markdown import markdown
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.base import Embeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from typing import List, Dict, Any, Tuple, Union
from rag_prompt import RAG_PROMPT
from dotenv import load_dotenv

# load environment variables
load_dotenv()

MARKDOWN_PATH = "data/markdown/markdown_data.json"
HTML_PATH = "data/html/html_data.json"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING_MODEL = "text-embedding-ada-002"
TOP_K = 5
OUTPUT_DIR = "vector_store"
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
MODEL_DIR = "models"

# Download documents
The RAG application should be able to answer questions based on ingested documents. For the SUTD chatbot, download PDF and HTML files from the SUTD website. The documents should contain information about the admission process, available courses and the university in general.


In [3]:
# QUESTION: Download documents from the SUTD website
# You should download at least 10 documents but more documents can increase the knowledge base of your chatbot.

# **--- ADD YOUR SOLUTION HERE (20 points) ---**
def get_data(path):
    with open(path, "r", encoding="utf-8") as f:
        output_json = json.load(f)
    return output_json

html_data = get_data(HTML_PATH)
markdown_data = get_data(MARKDOWN_PATH)

print(
    f"Successfully loaded {len(html_data)} HTML documents and {len(markdown_data)} Markdown documents"
)

markdown_files = glob("data/markdown/*.md")
print(f"Found {len(markdown_files)} Markdown files on disk")

docs_info = []
for doc in markdown_data:
    docs_info.append(
        {
            "Title": doc.get("title", "No Title"),
            "URL": doc.get("url", "No URL"),
            "Has Markdown": bool(doc.get("markdown", "").strip()),
        }
    )

docs_df = pd.DataFrame(docs_info)
display(docs_df.head(10))

if len(docs_df) > 10:
    print(f"... and {len(docs_df) - 10} more documents")

print("\nDocument Statistics:")
print(f"Total SUTD documents: {len(docs_df)}")
print(f"Documents with extracted markdown content: {docs_df['Has Markdown'].sum()}")

Successfully loaded 30 HTML documents and 30 Markdown documents
Found 27 Markdown files on disk


,Title,URL,Has Markdown
0,SUTD About page,https://www.sutd.edu.sg/about/,True
1,SUTD Contact page,https://www.sutd.edu.sg/contact-us/contact-sutd/,True
2,SUTD Home page,https://www.sutd.edu.sg/,False
3,SUTD Application Guide page,https://www.sutd.edu.sg/admissions/undergradua...,True
4,SUTD Appeal Guide page,https://www.sutd.edu.sg/admissions/undergradua...,True
5,SUTD Admission Requirements page,https://www.sutd.edu.sg/admissions/undergradua...,True
6,SUTD Masters information page 1,https://www.sutd.edu.sg/admissions/graduate/ma...,False
7,SUTD Masters information page 2,https://www.sutd.edu.sg/admissions/graduate/ma...,True
8,SUTD PHD information page,https://www.sutd.edu.sg/admissions/graduate/phd/,True
9,SUTD Academic Calendar,https://www.sutd.edu.sg/education/undergraduat...,True


... and 20 more documents

Document Statistics:
Total SUTD documents: 30
Documents with extracted markdown content: 27


# Split documents
Use LangChain to split the documents into smaller text chunks. 

In [4]:
# QUESTION: Use langchain to split the documents into chunks

# --- ADD YOUR SOLUTION HERE (20 points)---
def process_documents(data):
    processed_docs = []

    # take in json data (dictionary) and create a list of documents
    for item in data:
        # get the markdown content
        if not item.get("markdown"):
            continue

        # extract the metadata from the json
        metadata = {
            "title": item.get("title", ""),
            "url": item.get("url", ""),
            "description": item.get("description", ""),
            "pillar": extract_pillar(item.get("title", ""), item.get("url", "")),
        }

        # normalize headers
        content = normalize_headers(item.get("markdown", ""))

        # create langchain document object
        doc = Document(page_content=content, metadata=metadata)

        processed_docs.append(doc)
    return processed_docs


def extract_pillar(title, url):
    # extract each pillar to add to metadata to make it easier to provide context to LLM
    pillars = ["ISTD", "ESD", "EPD", "ASD", "DAI", "HASS", "SMT"]

    for pillar in pillars:
        if pillar in title or pillar.lower() in url.lower():
            return pillar
    # if no pillar is found, return 'General'
    return "General"


def normalize_headers(markdown_text):
    # add a space after # characters for proper header parsing
    markdown_text = re.sub(r"(#{1,6})([^#\s])", r"\1 \2", markdown_text)
    return markdown_text


def extract_markdown_hierarchy(markdown_text):
    # convert the markdown text to html
    html = markdown(markdown_text)
    soup = BeautifulSoup(html, "html.parser")

    sections = []
    current_section = {"title": "Root", "level": 0, "content": "", "parents": []}
    # list of header tags to look for
    heading_tags = ["h1", "h2", "h3", "h4", "h5", "h6"]

    # first pass: identify all headings and their levels
    headings = []
    for tag in soup.find_all(heading_tags):
        level = int(tag.name[1])
        headings.append(
            {
                "tag": tag,
                "title": tag.get_text().strip(),
                "level": level,
            }
        )

    # if there were no headings found, treat the entire document as one section
    if not headings:
        current_section["content"] = markdown_text
        return [current_section]

    # second pass: extract section content and build hierarchy
    for i, heading in enumerate(headings):
        # find content up to the next heading or end of document
        content_elements = []
        element = heading["tag"].next_sibling

        while element and (i == len(headings) - 1 or element != headings[i + 1]["tag"]):
            if element.name not in heading_tags:
                if hasattr(element, "get_text"):
                    content_elements.append(str(element))
            element = element.next_sibling

        # get the parent headings
        parent_titles = []
        for prev_heading in reversed(headings[:i]):
            if prev_heading["level"] < heading["level"]:
                parent_titles.insert(0, prev_heading["title"])

        # build section
        section = {
            "title": heading["title"],
            "level": heading["level"],
            "content": "".join(content_elements),
            "parents": parent_titles,
        }
        sections.append(section)
    return sections

# the markdowns kept in some internal URLs which are useful
def extract_internal_urls(content):
    # since the markdown has a few links to internal pages, we need to extract them
    # pattern to match markdown links
    pattern = r"\[.*?\]\((https?://www\.sutd\.edu\.sg/[^)]+)\)"
    urls = re.findall(pattern, content)

    # also check for HTML links if any HTML is embedded in the markdown
    if '<a href="' in content:
        html_pattern = r'<a href="(https?://www\.sutd\.edu\.sg/[^"]+)"'
        html_urls = re.findall(html_pattern, content)
        urls.extend(html_urls)

    return list(set(urls))


def process_document(doc):
    # extract the metadata from the document
    metadata = doc.metadata
    content = doc.page_content

    # extract markdown structure
    sections = extract_markdown_hierarchy(content)

    # process each section into a chunk
    chunks = []
    for section in sections:
        # skip very short sections
        if len(section["content"]) < 10 and section["level"] > 0:
            continue

        # get the full text for this section
        section_title = f"# {section['title']}" if section["level"] > 0 else ""
        section_text = f"{section_title}\n\n{section['content']}"

        # extract all urls internal to the section
        internal_urls = extract_internal_urls(section_text)

        # create the metadata
        enhanced_metadata = {
            **metadata,
            "section_title": section["title"],
            "parent_sections": section["parents"],
            "section_level": section["level"],
            "internal_links": internal_urls,
        }

        chunks.append({"text": section_text.strip(), "metadata": enhanced_metadata})

    return chunks


def refine_chunks(chunks, min_size=100, max_size=1000):
    refined_chunks = []
    current_text = ""
    current_metadata = None

    for chunk in chunks:
        if (
            len(chunk["text"]) < min_size
            and current_metadata
            and chunk["metadata"]["parent_sections"]
            == current_metadata["parent_sections"]
        ):
            current_text += "\n\n" + chunk["text"]
        else:
            if current_text:
                refined_chunks.append(
                    Document(page_content=current_text, metadata=current_metadata)
                )

            current_text = chunk["text"]
            current_metadata = chunk["metadata"]

    if current_text:
        refined_chunks.append(Document(page_content=current_text, metadata=current_metadata))

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_size, chunk_overlap=100, separators=["\n\n", "\n", ".", " ", ""]
    )
    print(refined_chunks[0])

    final_chunks = text_splitter.split_documents(refined_chunks)
    return final_chunks


processed_docs = process_documents(markdown_data)
print(f"Processed documents: {len(processed_docs)}")

all_chunks = []
for doc in processed_docs:
    doc_chunks = process_document(doc)
    all_chunks.extend(doc_chunks)

print(f"Total chunks created: {len(all_chunks)}")

refined_chunks = refine_chunks(all_chunks)
print(f"Total refined chunks: {len(refined_chunks)}")
print(type(refined_chunks))


Processed documents: 27
Total chunks created: 213
page_content='# About SUTD


<p>SUTD integrates design, AI and technology into a holistic, interdisciplinary education and research experience. This unique approach encourages our students to push the boundaries of innovating solutions to real-world problems.</p>' metadata={'title': 'SUTD About page', 'url': 'https://www.sutd.edu.sg/about/', 'description': 'Provides an overview of SUTD, its mission, and its unique educational approach.', 'pillar': 'General', 'section_title': 'About SUTD', 'parent_sections': [], 'section_level': 1, 'internal_links': []}
Total refined chunks: 270
<class 'list'>


In [5]:
# QUESTION: Use langchain to split the documents into chunks

# --- ADD YOUR SOLUTION HERE (20 points)---
def process_documents(data):
    processed_docs = []

    # take in json data (dictionary) and create a list of documents
    for item in data:
        # get the markdown content
        if not item.get("markdown"):
            continue

        # extract the metadata from the json
        metadata = {
            "title": item.get("title", ""),
            "url": item.get("url", ""),
            "description": item.get("description", ""),
            "pillar": extract_pillar(item.get("title", ""), item.get("url", "")),
        }

        # normalize headers
        content = normalize_headers(item.get("markdown", ""))

        # create langchain document object
        doc = Document(page_content=content, metadata=metadata)

        processed_docs.append(doc)
    return processed_docs


def extract_pillar(title, url):
    # extract each pillar to add to metadata to make it easier to provide context to LLM
    pillars = ["ISTD", "ESD", "EPD", "ASD", "DAI", "HASS", "SMT"]

    for pillar in pillars:
        if pillar in title or pillar.lower() in url.lower():
            return pillar
    # if no pillar is found, return 'General'
    return "General"


def normalize_headers(markdown_text):
    # add a space after # characters for proper header parsing
    markdown_text = re.sub(r"(#{1,6})([^#\s])", r"\1 \2", markdown_text)
    return markdown_text


def extract_markdown_hierarchy(markdown_text):
    # convert the markdown text to html
    html = markdown(markdown_text)
    soup = BeautifulSoup(html, "html.parser")

    sections = []
    current_section = {"title": "Root", "level": 0, "content": "", "parents": []}
    # list of header tags to look for
    heading_tags = ["h1", "h2", "h3", "h4", "h5", "h6"]

    # first pass: identify all headings and their levels
    headings = []
    for tag in soup.find_all(heading_tags):
        level = int(tag.name[1])
        headings.append(
            {
                "tag": tag,
                "title": tag.get_text().strip(),
                "level": level,
            }
        )

    # if there were no headings found, treat the entire document as one section
    if not headings:
        current_section["content"] = markdown_text
        return [current_section]

    # second pass: extract section content and build hierarchy
    for i, heading in enumerate(headings):
        # find content up to the next heading or end of document
        content_elements = []
        element = heading["tag"].next_sibling

        while element and (i == len(headings) - 1 or element != headings[i + 1]["tag"]):
            if element.name not in heading_tags:
                if hasattr(element, "get_text"):
                    content_elements.append(str(element))
            element = element.next_sibling

        # get the parent headings
        parent_titles = []
        for prev_heading in reversed(headings[:i]):
            if prev_heading["level"] < heading["level"]:
                parent_titles.insert(0, prev_heading["title"])

        # build section
        section = {
            "title": heading["title"],
            "level": heading["level"],
            "content": "".join(content_elements),
            "parents": parent_titles,
        }
        sections.append(section)
    return sections

# the markdowns kept in some internal URLs which are useful
def extract_internal_urls(content):
    # since the markdown has a few links to internal pages, we need to extract them
    # pattern to match markdown links
    pattern = r"\[.*?\]\((https?://www\.sutd\.edu\.sg/[^)]+)\)"
    urls = re.findall(pattern, content)

    # also check for HTML links if any HTML is embedded in the markdown
    if '<a href="' in content:
        html_pattern = r'<a href="(https?://www\.sutd\.edu\.sg/[^"]+)"'
        html_urls = re.findall(html_pattern, content)
        urls.extend(html_urls)

    return list(set(urls))


def process_document(doc):
    # extract the metadata from the document
    metadata = doc.metadata
    content = doc.page_content

    # extract markdown structure
    sections = extract_markdown_hierarchy(content)

    # process each section into a chunk
    chunks = []
    for section in sections:
        # skip very short sections
        if len(section["content"]) < 10 and section["level"] > 0:
            continue

        # get the full text for this section
        section_title = f"# {section['title']}" if section["level"] > 0 else ""
        section_text = f"{section_title}\n\n{section['content']}"

        # extract all urls internal to the section
        internal_urls = extract_internal_urls(section_text)

        # create the metadata
        enhanced_metadata = {
            **metadata,
            "section_title": section["title"],
            "parent_sections": section["parents"],
            "section_level": section["level"],
            "internal_links": internal_urls,
        }

        chunks.append({"text": section_text.strip(), "metadata": enhanced_metadata})

    return chunks


def refine_chunks(chunks, min_size=100, max_size=1000):
    refined_chunks = []
    current_text = ""
    current_metadata = None

    for chunk in chunks:
        if (
            len(chunk["text"]) < min_size
            and current_metadata
            and chunk["metadata"]["parent_sections"]
            == current_metadata["parent_sections"]
        ):
            current_text += "\n\n" + chunk["text"]
        else:
            if current_text:
                refined_chunks.append(
                    Document(page_content=current_text, metadata=current_metadata)
                )

            current_text = chunk["text"]
            current_metadata = chunk["metadata"]

    if current_text:
        refined_chunks.append(Document(page_content=current_text, metadata=current_metadata))

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_size, chunk_overlap=100, separators=["\n\n", "\n", ".", " ", ""]
    )
    print(refined_chunks[0])

    final_chunks = text_splitter.split_documents(refined_chunks)
    return final_chunks


processed_docs = process_documents(markdown_data)
print(f"Processed documents: {len(processed_docs)}")

all_chunks = []
for doc in processed_docs:
    doc_chunks = process_document(doc)
    all_chunks.extend(doc_chunks)

print(f"Total chunks created: {len(all_chunks)}")

refined_chunks = refine_chunks(all_chunks)
print(f"Total refined chunks: {len(refined_chunks)}")
print(type(refined_chunks))


Processed documents: 27
Total chunks created: 213
page_content='# About SUTD


<p>SUTD integrates design, AI and technology into a holistic, interdisciplinary education and research experience. This unique approach encourages our students to push the boundaries of innovating solutions to real-world problems.</p>' metadata={'title': 'SUTD About page', 'url': 'https://www.sutd.edu.sg/about/', 'description': 'Provides an overview of SUTD, its mission, and its unique educational approach.', 'pillar': 'General', 'section_title': 'About SUTD', 'parent_sections': [], 'section_level': 1, 'internal_links': []}
Total refined chunks: 270
<class 'list'>


### QUESTION: 

What chunking method or strategy did you use? Why did you use this method. Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**

We designed a hierarchical chunking strategy that respects the natural flow of a document. First, we extract logical sections using markdown headers (h1-h6), turning each section into its own chunk with its heading, content, and useful metadata like title, URL, parent sections, and pillar/department information. To avoid fragmentation, we combine very short chunks (under 100 characters) with nearby related content, and for long chunks (over 1000 characters), we split them using a recursive approach that breaks at natural separators such as paragraphs or sentences. We also keep a 100-character overlap between chunks to maintain context. This way, each chunk remains a complete, meaningful unit, perfectly sized for embedding and retrieval.

------------------------------


In [8]:
# The following code is the implementation for using LangChain to create a vector store
# But it didn't allow us to switch between different embeddings easily with various models

# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings

# embedding = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)
# retriever = FAISS.from_documents(refined_chunks, embedding).as_retriever(search_kwargs={"k": 7})

In [7]:
# QUESTION: create embeddings of document chunks and store them in a local vector store for fast lookup
# Decide an appropriate embedding model. Use Huggingface to run the embedding model locally.
# You do not have to use cloud-based APIs.

# --- ADD YOUR SOLUTION HERE (20 points)---
# QUESTION: create embeddings of document chunks and store them in a local vector store for fast lookup
# Decide an appropriate embedding model. Use Huggingface to run the embedding model locally.
# You do not have to use cloud-based APIs.
# --- ADD YOUR SOLUTION HERE (20 points)---



# creating a retriever class to make it compatible with langchain and allow for huggingface embeddings and openai embeddings
# this class that can handle both local Hugging Face models AND OpenAI models in one implementation (we made this to make it easy for us to switch between models)
# the best part is storing the embeddings in a local vector store (i think langchain has this as well but we made our own)
# we just added this layer on top of the simple langchain implementation
class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name = model_name
        if model_name.startswith("text-embedding-"):
            self.model = None
        else:
            self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        print(f"\nembedding {len(texts)} items with model: {self.model_name}")
        if self.model_name.startswith("text-embedding-"):
            response = openai.embeddings.create(model=self.model_name, input=texts)
            return [r.embedding for r in response.data]
        else:
            embeddings = self.model.encode(
                texts, convert_to_numpy=True, normalize_embeddings=True
            ).astype("float32")
            return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# using FAISS to create a vector store from the documents and using it's as_retriever method to create a retriever
# setting default k to 7
# embedding model chosen is text-embedding-ada-002
def create_retriever(refined_chunks, embedding_model="sentence-transformers/all-MiniLM-L6-v2", k=7):
    embeddings = CustomEmbeddings(model_name=embedding_model)

    if len(refined_chunks) > 0:
        print(f"example chunk: {refined_chunks[0]}")

    vectorstore = FAISS.from_documents(refined_chunks, embeddings)
    vector_store_dir = OUTPUT_DIR
    os.makedirs(vector_store_dir, exist_ok=True)
    vectorstore.save_local(vector_store_dir)

    print(f"saved vector store with {len(refined_chunks)} documents")
    return vectorstore.as_retriever(search_kwargs={"k": k})

embedding_model = EMBEDDING_MODEL
retriever = create_retriever(refined_chunks, embedding_model=embedding_model)

example chunk: page_content='# About SUTD


<p>SUTD integrates design, AI and technology into a holistic, interdisciplinary education and research experience. This unique approach encourages our students to push the boundaries of innovating solutions to real-world problems.</p>' metadata={'title': 'SUTD About page', 'url': 'https://www.sutd.edu.sg/about/', 'description': 'Provides an overview of SUTD, its mission, and its unique educational approach.', 'pillar': 'General', 'section_title': 'About SUTD', 'parent_sections': [], 'section_level': 1, 'internal_links': []}

embedding 270 items with model: text-embedding-ada-002
saved vector store with 270 documents


In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

# the idea here is to rerank those candidates with a more powerful model (slower but more accurate)
# we think that the retriever may not be able to pick the best documents hence use a reranker to pick the best documents
# this is done by using the FlashrankRerank class
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

### QUESTION: 

What embeddings and vector store did you use and why? Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**

We used OpenAI's "text-embedding-ada-002" for embeddings and FAISS (Facebook AI Similarity Search) as our vector store. The "text-embedding-ada-002" model provides embeddings with high quality semantic understanding of academic and technical content, which is important for accurately representing SUTD's educational information. We have compared the performance of this model against the following other models namely: all-MiniLM-L6-v2, all-mpnet-base-v2, and text-embedding-3-small. Among the 4, text-embedding-ada-002 was able to perform the best (based on human judgement) and it also integrates smoothly alongside the vector store chosen.

FAISS was our go-to choice because it efficiently handles nearest-neighbor searches in high-dimensional spaces, retrieves results quickly even from large collections, and keeps memory usage low through quantization. We have done research on other strategies such as Pinecone and Weaviate, but the ease of integration with FAISS utlimately helped us make our decision. Moreover, we have come across a lot of research in which FAISS was used, which proves its reliability.

We built a custom embeddings class that works with both OpenAI and local HuggingFace models, so switching between them is seamless while using the same interface. This setup delivers fast, accurate semantic search results while reliably keeping the vector store locally.

------------------------------



In [9]:
# Execute a query against the vector store

query = "When was SUTD founded?"

# QUESTION: run the query against the vector store, print the top 5 search results

#--- ADD YOUR SOLUTION HERE (5 points)---
# TODO: manually add in when SUTD was founded to the dataset
print(f"Query: {query}")

# use the query_index function to get the top k results
response = compression_retriever.invoke(query)
pretty_print_docs(response)
#------------------------------

Query: When was SUTD founded?

embedding 1 items with model: text-embedding-ada-002


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Document 1:

# About SUTD


<p>SUTD integrates design, AI and technology into a holistic, interdisciplinary education and research experience. This unique approach encourages our students to push the boundaries of innovating solutions to real-world problems.</p>
Metadata: {'id': 0, 'relevance_score': np.float32(0.99885666), 'title': 'SUTD About page', 'url': 'https://www.sutd.edu.sg/about/', 'description': 'Provides an overview of SUTD, its mission, and its unique educational approach.', 'pillar': 'General', 'section_title': 'About SUTD', 'parent_sections': [], 'section_level': 1, 'internal_links': []}
----------------------------------------------------------------------------------------------------
Document 2:

# Quick Links


<ul>
<li><a href="https://www.sutd.edu.sg/about/partnering-with-sutd/giving/">Donate to SUTD</a></li>
<li><a href="https://www.sutd.edu.sg/enterprise/research-collaborations/">Research collaboration</a></li>
<li><a href="https://www.sutd.edu.sg/enterprise/tech

## Huggingface Login

In [10]:
from huggingface_hub import login
login(token=HUGGINGFACE_TOKEN)

In [11]:
device = 0 if torch.cuda.is_available() else -1
print(device)

0


In [12]:
# QUESTION: Use the Huggingface transformers library to load the Llama 3.2-3B instruct model
# https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
# Run the model locally. You do not have to use cloud-based APIs.

# Execute the below query against the model and let it it answer from it's internal memory

query = "What courses are available in SUTD?"


#--- ADD YOUR SOLUTION HERE (40 points)---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-3B-Instruct"

pipeline = pipeline(
    "text-generation", model=model_id, max_new_tokens=256, device = device # setting max_new_tokens to 512 to be able to run on my GPU
)

# TODO: fix 
output = pipeline(query)
print("Model Response:")
print(output[0]["generated_text"])

#------------------------------

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Model Response:
What courses are available in SUTD??
Singapore University of Technology and Design (SUTD) offers a wide range of undergraduate and graduate courses across various disciplines. Here are some of the courses available in SUTD:

**Undergraduate Courses**

1. Bachelor of Science in Information Technology (BSIT)
2. Bachelor of Science in Information Systems (BSIS)
3. Bachelor of Science in Information Technology and Systems (BSITS)
4. Bachelor of Science in Data Science (BSDS)
5. Bachelor of Science in Artificial Intelligence and Data Science (BSAIDS)
6. Bachelor of Science in Human-Computer Interaction (BSHCI)
7. Bachelor of Science in Information Technology and Systems (BSITS)
8. Bachelor of Science in Computer Science (BSCS)
9. Bachelor of Science in Engineering (BSE)
10. Bachelor of Science in Design (BSD)
11. Bachelor of Science in Architecture (BSA)
12. Bachelor of Science in Biomedical Engineering (BSBME)
13. Bachelor of Science in Electrical Engineering (BSEE)
14. Bac

In [13]:
# QUESTION: Now put everything together. Use langchain to integrate your vector store and Llama model into a RAG system
# Run the below example question against your RAG system.

# Example questions
# TODO: what does this mean?
query = "How can I increase my chances of admission into SUTD?"


#--- ADD YOUR SOLUTION HERE (40 points)---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.float16,  # using float16 for faster inference
    device_map="auto",
    load_in_4bit=True,          # adding a 4-bit quantization
    low_cpu_mem_usage=True
)

pipeline = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256 # setting max_new_tokens to 512 to be able to run on my GPU
)


llm = HuggingFacePipeline(pipeline=pipeline)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    verbose=True,
)

result = rag_chain.run(query)
print("RAG Chain Response:")
print(result)
#------------------------------


OSError: models is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# QUESTION: Below is set of test questions. Add another 10 test questions based on your user interviews and your value proposition canvas.
# Run the complete set of test questions against the RAG question answering system.

questions = ["What are the admissions deadlines for SUTD?",
             "Is there financial aid available?",
             "What is the minimum score for the Mother Tongue Language?",
             "Do I require reference letters?",
             "Can polytechnic diploma students apply?",
             "Do I need SAT score?",
             "How many PhD students does SUTD have?",
             "How much are the tuition fees for Singaporeans?",
             "How much are the tuition fees for international students?",
             "Is there a minimum CAP?"
             ]

#--- ADD YOUR SOLUTION HERE (20 points)---
additional_questions = [
    "What is SUTD’s mission and vision?",
    "When was SUTD officially inaugurated?",
    "Which core values does SUTD emphasize?",
    "Where is SUTD located, and how can it be contacted?",
    "What different SUTD offices or departments can I reach out to?",
    "What are the key components of the Freshmore curriculum at SUTD?",
    "Which elective modules are available for Freshmore students in Term 3?",
    "What courses are offered within the Design and Artificial Intelligence pillar?",
    "Who are some of the instructors teaching the courses in the DAI program?",
    "What are the main steps involved in the SUTD application process?"
]

all_questions = questions + additional_questions
for q in all_questions:
    print("----------------------------------------------------------------")
    print("Question: " + q)
    #run the RAG chain
    result = rag_chain.run(q)
    print("Response:")
    print(result)
    print("----------------------------------------------------------------\n")#---------------------------

### QUESTION: 


Manually inspect each answer, fact check whether the answer is correct (use Google or any other method) and check the retrieved documents

For each question, answer and context triple, record the following

- How accurate is the answer (1-5, 5 best)?
- How relevant is the retrieved context (1-5, 5 best)?
- How grounded is the answer in the retrieved context (instead of relying on the LLM's internal knowledge) (1-5, 5 best)?

**--- ADD YOUR SOLUTION HERE (20 points) ---**


------------------------------



You can try improve the chatbot by going back to previous steps in the notebook and change things until the submission deadline. For example, you can add more data sources, change the embedding models, change the data pre-processing, etc. 


# End

This concludes assignment 3.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_03_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered. 
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions



**Assignment due 6 April 2025 11:59pm**